In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5, channel_sizes=None):
        super().__init__()
        if channel_sizes == None:
            channel_sizes = [6, 16]
        self.channel_sizes = channel_sizes
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, channel_sizes[0], kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channel_sizes[0], channel_sizes[1], kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = channel_sizes[1] * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
class Net3Conv(nn.Module):
    def __init__(self, kernel_size=5, channel_sizes=None):
        super().__init__()
        if channel_sizes == None:
            channel_sizes = [8, 16, 32]
        self.channel_sizes = channel_sizes
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, channel_sizes[0], kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channel_sizes[0], channel_sizes[1], kernel_size)
        self.conv3 = nn.Conv2d(channel_sizes[1], channel_sizes[2], kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = channel_sizes[2] * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [3]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    device = None # Wybiera CUDA jeśli jest dostępne, w przeciwnym wypadku CPU
    # device = "cpu" # Odkomentować jeżeli CUDA nie będzie działać
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, device=device, print_results=True)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer=None, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
        del net
        torch.cuda.empty_cache()
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [4]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]
transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))

dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return Net3Conv(channel_sizes=[16,16,16])

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Epoch 0:
	train loss: 2.127503083050762
	validation loss: 1.8147951887078482, validation accuracy: 38.6986301369863%
	Elapsed time: 0:00:12.739831
Epoch 1:
	train loss: 1.3873731662642281
	validation loss: 1.2808435943028698, validation accuracy: 57.1917808219178%
	Elapsed time: 0:00:12.952635
Epoch 2:
	train loss: 0.9480015882905924
	validation loss: 1.1788051765258998, validation accuracy: 59.93150684931506%
	Elapsed time: 0:00:13.042410
Epoch 3:
	train loss: 0.7428801191609679
	validation loss: 1.2793030297919497, validation accuracy: 61.3013698630137%
	Elapsed time: 0:00:12.685100
Epoch 4:
	train loss: 0.6060818283472013
	validation loss: 1.0206948763703647, validation accuracy: 68.4931506849315%
	Elapsed time: 0:00:12.821887
Epoch 5:
	train loss: 0.4038067617506351
	validation loss: 1.1403333601886279, validation accuracy: 67.12328767123287%
	Elapsed time: 0:00:12.898829
Epoch 6:
	train loss: 0.35334507852639385
	validation loss: 1.1707068012185293, validati

KeyboardInterrupt: 

In [ ]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/net_3conv_2fc_channels_16_16_16.csv")